In [1]:
# Import relevant libraries:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

from sklearn import metrics
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error as mse


In [2]:
# The goal is to obtain a predictor model for the amount of rain in a given month. The predictors are previous 
# months by order of less rainy to most and temperature, which is expected to have a negative correlation

In [3]:
# Open dataset:

df = pd.read_csv("Temp+Prec")

In [4]:
# As our predictor variable, we will take months but reordered by the months that have seen the most rain 
# historically. In that way, by restarting the indexes the x axis will make sense: 0 will be July, the driest month,
# and 11 will be October, the wettest. Hence, we will be measuring the chance of extreme rain as the months become
# rainier, since analysing with months in their natural order would make no sense in a regression

by_month = df[["Month", "Precipitation"]]

added_m = by_month.groupby("Month").aggregate(sum)

added_m.sort_values(by="Precipitation", ascending=True, inplace=True)

added_m.head(12)

,Precipitation
Month,
7,6012.4
2,8068.4
6,8536.2
1,8755.7
8,9041.3
12,9929.2
3,11119.1
4,12402.2
5,12454.7


In [5]:
# Convert months into ranking of accumulated rain:

df["independent_var"] = df["Month"].copy()

old_months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

new_months = [4, 2, 7, 8, 9, 3, 1, 5, 11, 12, 10, 6]

df["independent_var"] = df["independent_var"].replace(old_months, new_months)

In [6]:
# Create X and y variables and split them

X = df[["independent_var", "Temperature"]]

y = df["Precipitation"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [7]:
# Start with Random Forest Regressor model:

rf = RandomForestRegressor(n_estimators = 100, random_state = 42)

rf.fit(X_train, y_train)

Y_pred = rf.predict(X_test)

rf.feature_importances_

# Temperature seems to be the most important predictor

array([0.32150014, 0.67849986])

In [8]:
print("R^2 score: " + str(rf.score(X_train, y_train)))
print("R^2 score: " + str(rf.score(X_test, y_test)))

print("RMSE score: " + str(np.sqrt(mse(y_test, Y_pred))))

# These indexes point towards a non-relationship between the variables in study

R^2 score: 0.3736211185468865
R^2 score: -0.13782470001727964
RMSE score: 48.232988208363054


In [9]:
# Now a simple linear regression is tried. First we add a constant:

X_train = sm.add_constant(X_train) 
X_test = sm.add_constant(X_test) 

In [10]:
# Then we instantiate the model:

model = sm.OLS(y_train, X_train).fit()

predictions = model.predict(X_test)

model.summary()

# The results could indicate a correlation between the target value and the predictors; however, the R^2 is very
# low, indicating an imperfect. As a result, a correlation between variables cannot be assured.

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Precipitation   R-squared:                       0.111
Model:                            OLS   Adj. R-squared:                  0.110
Method:                 Least Squares   F-statistic:                     128.3
Date:                Wed, 22 Sep 2021   Prob (F-statistic):           3.14e-53
Time:                        17:04:00   Log-Likelihood:                -10650.
No. Observations:                2061   AIC:                         2.131e+04
Df Residuals:                    2058   BIC:                         2.132e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              16.7318      3.209      5.214      0.000      10.438      23.025
independent_var     4.3203      0.271     15.957      0.000       3.789       4.851
Temperature         0.2919      0.170      1.712      0.087      -0.042       0.626
==============================================================================
Omnibus:                      857.208   Durbin-Watson:                   1.915
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4875.120
Skew:                           1.884   Prob(JB):                         0.00
Kurtosis:                       9.524   Cond. No.                         57.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
rmse = np.sqrt(model.mse_total)

print("RMSE: " + str(rmse))

# Quite high value of unexplained variance: overall, a not good enough model

RMSE: 45.03933949296932


In [ ]:
# Conclusions:

    # The results for these models are not significant enough to state that there is a correlation between 
    # temperature or month of the year and precipitation. More data has to be added to the
    # dataset so that other variables can be studied. 
    
    # However, the conclusions we can extract from the descriptive analysis are that the rainiest seasons are 
    # Autumn and Fall and that overall the amount of rain in a year is larger now than during the XIXth Century. 
    # In terms of high temperatures, plenty of the most extreme cases have happened in the first two decades of the
    # XXIst Century.
    
    

In [ ]:
# Instantiate the Logistic Regression Model:

logreg = LogisticRegression(class_weight="balanced")

logreg.fit(X_train,y_train)

y_pred=logreg.predict(X_test)

In [ ]:
# Confusion matrix:

cnf_matrix = metrics.confusion_matrix(y_test, y_pred)

cnf_matrix

# 54 true positives and 404 true negatives; 247 cases incorrectly classified

In [ ]:
# AUC score:

y_pred_prob = logreg.predict_proba(X_test)[:, 1] 

metrics.roc_auc_score(y_test, y_pred_prob)

# Quita bad AUC score, it would be more efficient to detect all cases as non-rainy

In [ ]:
# Now use of a SVC algorithm instead to see if it improves performance:

vector = svm.SVC(kernel='linear', probability=True, class_weight="balanced")

vector.fit(X_train, y_train)

y_pred = vector.predict(X_test)

In [ ]:
# Confusion matrix:

cnf_matrix = metrics.confusion_matrix(y_test, y_pred)

cnf_matrix

# Same results as logistic regression

In [ ]:
# AUC score:

y_pred_prob = logreg.predict_proba(X_test)[:, 1] 

metrics.roc_auc_score(y_test, y_pred_prob)

In [ ]:
# Interpretation of the results: 

    # If we take a period of 2 years approximately (705 days, size of y_test), by only being based on the month 
    # of the year, we could detect 54 out of 85 very rainy days. This would come at the cost of misclassifying 247
    # days
    
    # It appears quite obvious that this method or approach has been insufficient. 
    
    

In [ ]:
# We already saw on Descriptive Analysis that 75% of months rains 69,1 l/m^2 or less. We will make a target value
# with this mark as the threshold

def rain_thresh(rain):
    if rain <= 69.1:
        return 0
    else:
        return 1

df["target"] = df["Precipitation"].apply(rain_thresh)

In [ ]:
df[df["target"] == 1].describe() 

# If we examine the values within the highest 25% of rainy days (which probably correspond to Autumn days), we see
# more inequality among values; to get the extreme values, we will analyse values that are higher than 100 l/m^2

In [ ]:
# We reapply the previous function with the threshold increased:

def rain_extreme(rain):
    if rain <= 100:
        return 0
    else:
        return 1

df["target"] = df["Precipitation"].apply(rain_extreme)